In [14]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json 
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import re
import gensim
from gensim import corpora

import spacy
# spacy.prefer_gpu()

import joblib

In [2]:
PATH_INPUT = 'pruebas/'
FILE_INPUT = 'pruebas.xlsx'
PATH_OUTPUT = 'pruebas/'
FILE_OUTPUT = 'pruebas.json'

LDA_MODEL = 'models/lda_model.jl'
LDA_DICT = 'models/lda_diccionario.jl'
SVM_MODEL = 'models/svm_model.jl'

In [18]:
# Leyendo archivos
path = PATH_INPUT+FILE_INPUT
print('Leyendo:',path)
data = read_file_xlsx(path)

# Corecciones primarias
path_geodata = 'src/dict/BaseCentrosPoblados.csv'
data = primary_updates(data, path_geodata)

# Uniones con catalogos
pc_bn = 'src/catalogs/catalogo_BN.csv'
pc_sbs = 'src/catalogs/catalogo_SBS.xlsx'
pc_nivel = 'src/catalogs/catalogo_NIVEL.xlsx'
paths = [pc_bn, pc_sbs, pc_nivel]
data = catalogs_joiner(data, paths)

# Limpieza de texto
global nlp
nlp = spacy.load('es_core_news_sm', disable=['parser', 'ner'])

dict_LDA = joblib.load(LDA_DICT)
corpus = text_process_lema(data, dict_LDA)

Leyendo: pruebas/pruebas.xlsx
Dataset corregido y renombrado ...
Leyendo: src/catalogs/catalogo_BN.csv
Leyendo: src/catalogs/catalogo_SBS.xlsx
Leyendo: src/catalogs/catalogo_NIVEL.xlsx
Uniendo catalogos...
Procesando textos (descripciones)...
Elementos nulos 4
Elementos nulos 0


In [19]:



# Computed by LDA model
model_LDA = joblib.load(LDA_MODEL)

counter = 0
features = []
for values, index, score in sorted(model_LDA[corpus]):
    abc = [i[1] for i in values]
    if len(abc)<4:
        out = [0,0,0,0]
        out[:len(abc)] = abc
        abc = out
    features.append(abc)
    counter += 1
#df_results = pd.DataFrame(features, columns = ['A', 'B', 'C', 'D'])

# Computed by SVM model
model_SVM = joblib.load(SVM_MODEL)
data['TAG'] = model_SVM.predict(features)

# Filtrando AT
data = data[data['TAG']==1]
data = data[['Nombre','Direccion domicilio','Departamento','Numero reclamo','Canal Incidencia','Correo electronico','fecha fase 1', 'Servicio', 'Motivo']]


In [33]:
abc = data.to_dict('index')
for key in abc:
    print(abc[key])
    print()

{'Nombre': 'BRINER STIVE SANCHEZ ESPINOZA', 'Direccion domicilio': 'AA.HH. NUEVO AMANECER MZ U LT 5', 'Departamento': 'LIMA', 'Numero reclamo': '002721-45822', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'briner_16@hotmail.com', 'fecha fase 1': Timestamp('2021-06-25 00:00:00'), 'Servicio': 'CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMUNERACIONES)', 'Motivo': 'INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, POLIZAS, CONDICIONES, ACUERDOS'}

{'Nombre': 'DIAZ ZAMORA YENEFER', 'Direccion domicilio': 'JIRON VILLA UMU 1331 URB ZARATE', 'Departamento': 'LIMA', 'Numero reclamo': '006221-82', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'YENEFER_IDA@HOTMAIL.COM', 'fecha fase 1': Timestamp('2021-07-30 00:00:00'), 'Servicio': 'CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMUNERACIONES)', 'Motivo': 'OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS,DISPOSICIONES,RETIROS,CARGOS Y ABONOS) '}

{'Nombre': 'PLACENCIA CISNEROS ANTONIO', 'Direccion domicilio': 'JR CHINCHAYSUYO 631  URB ZARATE'


{'Nombre': 'MIRIAM REYES ESPINOZA', 'Direccion domicilio': 'URB.LOS SAUCES MZ, D1 LT.31', 'Departamento': 'LAMBAYEQUE', 'Numero reclamo': '002721-46881', 'Canal Incidencia': 'No existe un canal asociado', 'Correo electronico': 'mirlen20.mr@gmail.com', 'fecha fase 1': Timestamp('2021-07-05 00:00:00'), 'Servicio': 'TARJETA DE DEBITO - VISA', 'Motivo': 'TRANSACCION NO PROCESADA / MAL REALIZADA'}

{'Nombre': 'PORTAL PEREZ CESAR LUIS GILBERTO', 'Direccion domicilio': 'PSJE. BELLAVISTA N°206', 'Departamento': 'CAJAMARCA', 'Numero reclamo': '315121-4330', 'Canal Incidencia': 'No existe un canal asociado', 'Correo electronico': 'cesar_luis_p.p@hotmail.com', 'fecha fase 1': Timestamp('2021-07-05 00:00:00'), 'Servicio': 'TARJETA DE DEBITO - VISA', 'Motivo': 'TRANSACCION NO PROCESADA / MAL REALIZADA'}

{'Nombre': 'ERICK JARA BENAVIDES', 'Direccion domicilio': 'AV MEXICO 444 - COMAS', 'Departamento': 'LIMA', 'Numero reclamo': '002721-46836', 'Canal Incidencia': 'No existe un canal asociado', 'Cor

{'Nombre': 'GIANCARLO VILLENA PRADO', 'Direccion domicilio': 'Avenida Brasil 657', 'Departamento': 'LIMA', 'Numero reclamo': '002721-31436', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'giancarlo.villena18@gmail.com', 'fecha fase 1': Timestamp('2021-03-11 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA O INSUFICIENTE INFORMACION SOBRE OPERACIONES, PRODUCTOS Y/O SERVICIOS'}

{'Nombre': 'HELMER YUPANQUI QUISPE', 'Direccion domicilio': 'Mzi2 lt4, san genaro', 'Departamento': 'LIMA', 'Numero reclamo': '002721-31596', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'helmer.yupanqui@gmail.com', 'fecha fase 1': Timestamp('2021-03-11 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA O INSUFICIENTE INFORMACION SOBRE OPERACIONES, PRODUCTOS Y/O SERVICIOS'}

{'Nombre': 'municipalidad distrital de catac', 'Direccion domicilio': 'jr 8 de enero n° 361', 'Departamento': 'ANCASH', 'Numero reclamo': '002721-31424', 'Canal Inci


{'Nombre': 'PERALTA VEGA TANIA ALICIA', 'Direccion domicilio': 'CALLE LA MAR 809', 'Departamento': 'ICA', 'Numero reclamo': '060121-165', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'tpv2003@hotmail.com', 'fecha fase 1': Timestamp('2021-05-24 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA ATENCION AL USUARIO'}

{'Nombre': 'PEREZ OCHANTE ANA MARIA', 'Direccion domicilio': 'MZ.O LT.9 CMT.21', 'Departamento': 'LIMA', 'Numero reclamo': '001021-33', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'integracion.online1@gmail.com', 'fecha fase 1': Timestamp('2021-05-24 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA ATENCION AL USUARIO'}

{'Nombre': 'RIOS HUAMANI ESTHER ELIZABETH', 'Direccion domicilio': 'CA.LAS ESMERALDAS  585-A -BALCONCILLO', 'Departamento': 'LIMA', 'Numero reclamo': '220221-385', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'echilenari@gmail.com', 'fecha fase 1': Timestamp('2021-05-24 

{'Nombre': 'IGNACIO ARIZAPANA EDITH ROXANA', 'Direccion domicilio': 'JR. LORETO 1241', 'Departamento': 'JUNIN', 'Numero reclamo': '038621-9', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'glissyta@hotmail.com', 'fecha fase 1': Timestamp('2021-02-05 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA ATENCION AL USUARIO'}

{'Nombre': 'JOSE LUIS ALMEYDA ALMEYDA', 'Direccion domicilio': nan, 'Departamento': 'LIMA', 'Numero reclamo': '002721-12280', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'JALMEYDA395@GMAIL.COM', 'fecha fase 1': Timestamp('2021-02-05 00:00:00'), 'Servicio': 'SERVICIO DE ATENCION AL USUARIO', 'Motivo': 'INADECUADA ATENCION AL USUARIO'}

{'Nombre': 'LUIS SOTO VELASCO', 'Direccion domicilio': 'Barrio San Miguel s/n', 'Departamento': 'HUANUCO', 'Numero reclamo': '002721-12356', 'Canal Incidencia': 'OFICINA', 'Correo electronico': 'luissotovelasco7@gmail.com', 'fecha fase 1': Timestamp('2021-02-05 00:00:00'), 'Servicio': 'SERVIC


{'Nombre': 'CLELIA HERNANDEZ CARDENAS', 'Direccion domicilio': 'JR. CESAR VALLEJO 384', 'Departamento': 'HUANUCO', 'Numero reclamo': '002721-47868', 'Canal Incidencia': nan, 'Correo electronico': 'chanita1601@hotmail.com', 'fecha fase 1': Timestamp('2021-07-12 00:00:00'), 'Servicio': nan, 'Motivo': nan}

{'Nombre': 'MOISES TAPIA CHACOLLA', 'Direccion domicilio': 'Calle santa Beatriz 2041 - Natividad', 'Departamento': 'TACNA', 'Numero reclamo': '002721-47657', 'Canal Incidencia': nan, 'Correo electronico': 'tapiasmoises3@gmail.com', 'fecha fase 1': Timestamp('2021-07-10 00:00:00'), 'Servicio': nan, 'Motivo': nan}

{'Nombre': 'JUAN LOYOLA LA MADRID', 'Direccion domicilio': 'jr. independencia 337', 'Departamento': 'LIMA', 'Numero reclamo': '002721-47527', 'Canal Incidencia': nan, 'Correo electronico': 'jmcaspi@gmail.com', 'fecha fase 1': Timestamp('2021-07-09 00:00:00'), 'Servicio': nan, 'Motivo': nan}

{'Nombre': 'NAREZO HUALPA LUIS JULIAN', 'Direccion domicilio': 'JR. INTIHUATANA 262',

In [29]:

data = data[['Nombre','Direccion domicilio','Departamento','Numero reclamo','Canal Incidencia','Correo electronico','fecha fase 1', 'Servicio', 'Motivo']]
data.head()

,Nombre,Direccion domicilio,Departamento,Numero reclamo,Canal Incidencia,Correo electronico,fecha fase 1,Servicio,Motivo
0,BRINER STIVE SANCHEZ ESPINOZA,AA.HH. NUEVO AMANECER MZ U LT 5,LIMA,002721-45822,OFICINA,briner_16@hotmail.com,2021-06-25,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ..."
18,DIAZ ZAMORA YENEFER,JIRON VILLA UMU 1331 URB ZARATE,LIMA,006221-82,OFICINA,YENEFER_IDA@HOTMAIL.COM,2021-07-30,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS...
28,PLACENCIA CISNEROS ANTONIO,JR CHINCHAYSUYO 631 URB ZARATE,LIMA,006221-81,OFICINA,PLATA_A3@HOTMAIL.COM,2021-07-30,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS...
64,FERNANDEZ SALVATIERRA LIZ KATHERIN,ASENT.H. CRUZ DE MOTUPE GRUPO 7 MZ LT. 20,LIMA,006921-46,OFICINA,lizfernandezsalvatierra@gmail.com,2021-07-24,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS...
77,MEJIA VALDIVIA NAISSA AMPARO,URB LA ESPERANZA G 5 JOSE LUIS BUSTAMANTE Y RI...,AREQUIPA,010221-52,OFICINA,naissa.mejia@gmail.com,2021-07-23,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS...


In [30]:
data.to_dict('index')

{0: {'Nombre': 'BRINER STIVE SANCHEZ ESPINOZA',
  'Direccion domicilio': 'AA.HH. NUEVO AMANECER MZ U LT 5',
  'Departamento': 'LIMA',
  'Numero reclamo': '002721-45822',
  'Canal Incidencia': 'OFICINA',
  'Correo electronico': 'briner_16@hotmail.com',
  'fecha fase 1': Timestamp('2021-06-25 00:00:00'),
  'Servicio': 'CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMUNERACIONES)',
  'Motivo': 'INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, POLIZAS, CONDICIONES, ACUERDOS'},
 18: {'Nombre': 'DIAZ ZAMORA YENEFER',
  'Direccion domicilio': 'JIRON VILLA UMU 1331 URB ZARATE',
  'Departamento': 'LIMA',
  'Numero reclamo': '006221-82',
  'Canal Incidencia': 'OFICINA',
  'Correo electronico': 'YENEFER_IDA@HOTMAIL.COM',
  'fecha fase 1': Timestamp('2021-07-30 00:00:00'),
  'Servicio': 'CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMUNERACIONES)',
  'Motivo': 'OPERACIONES EN CUENTA NO RECONOCIDAS (CONSUMOS,DISPOSICIONES,RETIROS,CARGOS Y ABONOS) '},
 28: {'Nombre': 'PLACENCIA CISNEROS ANTONIO',
  'Direcci

In [15]:
result = data.head().to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed) 

'{"0": {"Nombre": "BRINER STIVE SANCHEZ ESPINOZA", "Direccion domicilio": "AA.HH. NUEVO AMANECER MZ U LT 5", "Departamento": "LIMA", "Numero reclamo": "002721-45822", "Canal Incidencia": "OFICINA", "Correo electronico": "briner_16@hotmail.com", "fecha fase 1": 1624579200000, "Servicio": "CUENTA DE AHORRO (DESTINADA A DEP\\u00d3SITO DE REMUNERACIONES)", "Motivo": "INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, POLIZAS, CONDICIONES, ACUERDOS"}, "1": {"Nombre": "SALAZAR HUANCA HUGO DIONY", "Direccion domicilio": "JR. AEROPUERTO 266 URB SAN JOSE II ETAPA", "Departamento": "PUNO", "Numero reclamo": "072121-54", "Canal Incidencia": "OFICINA", "Correo electronico": "hdsh@outlook.com", "fecha fase 1": 1618185600000, "Servicio": "CUENTA DE AHORRO (DESTINADA A DEP\\u00d3SITO DE REMUNERACIONES)", "Motivo": "INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, POLIZAS, CONDICIONES, ACUERDOS"}, "2": {"Nombre": "ARAGON CERVANTES DE VILLAVERDE ANA DE LOS ANGELES FERDINANDA", "Direccion domicilio": "CA. MAXIM

## Functions

In [3]:
def read_file_xlsx(path):
    data = pd.read_excel(path, index_col=0, header=0)
    return data

def similars(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return resp
    if max_val >= 0.8:
        return resp
    return text

def similars2(text, lista):
    max_val = 0
    resp = ''
    for e in lista:
        s = SequenceMatcher(None, str(text), e)
        if s.ratio() > max_val:
            max_val = s.ratio()
            resp = e
        if max_val == 1:
            return max_val
    return max_val

def primary_updates(data, path_geodata):
    geo_data = pd.read_csv(path_geodata, delimiter=';')
    
    geo_data = geo_data.groupby(['DEPARTAMEN','PROVINCIA','DISTRITO']).agg({ 'POB': "sum"})
    geo_data = geo_data.sort_values(by=['DEPARTAMEN','PROVINCIA','DISTRITO']).reset_index()

    lista = list(np.unique(geo_data['DEPARTAMEN']))
    data['DEPARTAMEN'] = data['Departam. Domic.'].apply(similars, args=(lista,))

    lista = list(np.unique(geo_data[geo_data['DEPARTAMEN']=='LIMA']['PROVINCIA']))
    data['PROVINCIA'] = data.apply(lambda x: similars(x['Provincia Domic.'], lista) if x['Departam. Domic.']=='LIMA' else x['Provincia Domic.'],axis=1)

    lista = list(np.unique(geo_data[(geo_data['DEPARTAMEN']=='LIMA') & (geo_data['PROVINCIA']=='LIMA')]['DISTRITO']))
    data['DISTRITO'] = data.apply(lambda x: similars(x['Distrito Domic.'], lista)  if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else x['Distrito Domic.'],axis=1)
    data['DISTRITO_VAL'] = data.apply(lambda x: similars2(x['Distrito Domic.'], lista) if x['Departam. Domic.']=='LIMA' and x['Provincia Domic.']=='LIMA' else 0,axis=1)

    data['IS_LIMA'] = data.apply(lambda x: True if x['DISTRITO_VAL']>=0.8 else False, axis=1)

    del data['DISTRITO_VAL']
    del data['Departam. Domic.']
    del data['Provincia Domic.']
    del data['Distrito Domic.']

    data['fecha_fase1'] = pd.to_datetime(data['Fase1_presen'], format='%d/%m/%Y')
    data['fecha_fase2'] = pd.to_datetime(data['TEMPORAL'], format='%d/%m/%Y')
    data['fecha_fase3'] = pd.to_datetime(data['CALIFFINAL'], format='%d/%m/%Y')
    data['fecha_fase4'] = pd.to_datetime(data['Fase4_generac'], format='%d/%m/%Y')
    data['fecha_fase5'] = pd.to_datetime(data['Fase5_revisac'], format='%d/%m/%Y')
    data['fecha_fase6'] = pd.to_datetime(data['Fase6_enviac'], format='%d/%m/%Y')
    data['fecha_fase7'] = pd.to_datetime(data['Fase7_firma_ACUSE'], format='%d/%m/%Y')

    del data['Fase1_presen']
    del data['TEMPORAL']
    del data['CALIFFINAL']
    del data['Fase4_generac']
    del data['Fase5_revisac']
    del data['Fase6_enviac']
    del data['Fase7_firma_ACUSE']

    data['Cod. Servicio'] = data['Cod. Servicio'].fillna(-1).astype(int).astype(str)
    data['Cod Motivo'] = data['Cod Motivo'].fillna(-1).astype(int).astype(str)

    data.rename(columns={
        'N° Reclamo': 'Numero reclamo',
        'Nombre': 'Nombre',
        'N° Docum.': 'Numero documento',
        'Cod. Servicio': 'Cod servicio',
        'Servicio': 'Servicio',
        'Cod Motivo': 'Cod motivo',
        'Motivo': 'Motivo',
        'Cod. Atm': 'Cod ATM',
        'Det. AtmError': 'ATM error',
        'N° Cajero': 'Numero cajero',
        'Monto': 'Monto',
        'Operac.': 'Fecha operacion',
        'Present.': 'Fecha presentacion',
        'Resul.': 'Fecha resultado',
        'Recl.': 'Estado reclamo',
        'Cod. Oficina': 'Cod oficina',
        'Det. Oficina': 'Det oficina',
        'Regional': 'Cod region',
        'Dependencia': 'Dependencia',
        'Cod. Deriv': 'Cod dependencia derivada',
        'Última Derivació': 'Ultima derivacion',
        'N° Memo Carta Resp.': 'Numero carta respuesta',
        'Dirección Domic.': 'Direccion domicilio',
        'Fase del Reclamo': 'Fase reclamo',
        'Proceso': 'error_1',
        'Días de Demora': 'error_2',
        ' de Demora': 'error_3',
        'Respon': 'error_4',
        'Responsable': 'error_5',
        'Macroregión': 'error_6',
        'Resp.': 'error_7',
        'División': 'error_8',
        'División Responsable': 'error_9',
        'Cod. Dept': 'error_10',
        'Responsable.1': 'error_11',
        'Mes Presentacion': 'Mes presentacion',
        'Días Reclamo': 'Dias reclamo',
        'Año Presentación': 'Anio presentacion',
        'Semáforo': 'error_12',
        'Situación': 'error_13',
        'Cod. Usuario': 'Cod usuario',
        'Nombre Usuario': 'Usuario',
        'Descripción Fase': 'error_14',
        'Canal Ingreso': 'Canal ingreso',
        'Medio Respuesta': 'Medio respuesta',
        'Reclamo/requerim': 'Categoria',
        'Hace carta': 'Hace carta',
        'Correo': 'Correo electronico',
        'Cta. Ahorro': 'Cuenta ahorro',
        'Descripcion': 'Descripcion',
        'DEPARTAMEN': 'Departamento',
        'PROVINCIA': 'Provincia',
        'DISTRITO': 'Distrito',
        'IS_LIMA': 'Es Lima',
        'fecha_fase1': 'fecha fase 1',
        'fecha_fase2': 'fecha fase 2',
        'fecha_fase3': 'fecha fase 3',
        'fecha_fase4': 'fecha fase 4',
        'fecha_fase5': 'fecha fase 5',
        'fecha_fase6': 'fecha fase 6',
        'fecha_fase7': 'fecha fase 7'
    }, inplace = True)

    columns = data.columns
    news_columns = []
    for i in columns:
        if 'error_' in i:
            pass
        else:
            news_columns.append(i)

    data = data[news_columns]

    del data['Motivo']
    del data['Servicio']
    
    print('Dataset corregido y renombrado ...')
    return data
 
def nivel(canal,servicio,motivo):
    try:
        canal = int(canal)
        return niveles['canales'][canal]['Nivel']
    except:
        pass
    try:
        servicio = int(servicio)
        return niveles['servicios'][servicio]['Nivel']
    except:
        pass
    try:
        motivo = int(motivo)
        return niveles['motivos'][motivo]['Nivel']
    except:
        pass
    return '2do'

In [4]:
def catalogs_joiner(data, paths):
    path_catalogo_bn = paths[0]
    path_catalogo_sbs = paths[1]
    path_catalogo_nivel = paths[2]
    # Catalogos BN
    schema = {
        'Servicio BN':int,
        'Motivo BN':int,
        'Canal Incidencia SBS': str,
        'Servicio SBS': str,
        'Motivo SBS': str,
    }
    catalogo_bn = pd.read_csv(path_catalogo_bn, dtype=schema)
    catalogo_bn['Servicio BN'] = catalogo_bn['Servicio BN'].astype(str)
    catalogo_bn['Motivo BN'] = catalogo_bn['Motivo BN'].astype(str)
    print('Leyendo:',path_catalogo_bn)

    # Catalogos SBS
    catalogo_sbs_motivo = pd.read_excel(path_catalogo_sbs, sheet_name='MOTIVOS')
    catalogo_sbs_servicios = pd.read_excel(path_catalogo_sbs, sheet_name='SERVICIOS')
    catalogo_sbs_canales = pd.read_excel(path_catalogo_sbs, sheet_name='CANALES')
    print('Leyendo:',path_catalogo_sbs)

    # Catalogos NIVEL
    catalogo_nivel_motivo = pd.read_excel(path_catalogo_nivel, sheet_name='MOTIVOS')
    catalogo_nivel_servicios = pd.read_excel(path_catalogo_nivel, sheet_name='SERVICIOS')
    catalogo_nivel_canales = pd.read_excel(path_catalogo_nivel, sheet_name='CANALES')
    print('Leyendo:',path_catalogo_nivel)

    niveles = {}
    niveles['servicios'] = catalogo_nivel_servicios.set_index('Cod_SBS').T.to_dict('dict')
    niveles['motivos'] = catalogo_nivel_motivo.set_index('Cod_SBS').T.to_dict('dict')
    niveles['canales'] = catalogo_nivel_canales.set_index('Cod_SBS').T.to_dict('dict')

    print('Uniendo catalogos...')
    datos_preprocessed_merged = pd.merge(
                                data,
                                catalogo_bn,
                                how="left",
                                left_on=['Cod servicio', 'Cod motivo'],
                                right_on=['Servicio BN', 'Motivo BN'],
                                sort=True,
                                suffixes=("_x", "_y"),
                            )
    datos_preprocessed_merged['Nivel'] = datos_preprocessed_merged.apply(lambda x: nivel(x['Canal Incidencia SBS'], x['Servicio SBS'], x['Motivo SBS']), axis=1)
    
    return datos_preprocessed_merged

In [5]:

def texto_a_palabras(texto:str):
    for sentencia in texto:
        yield(str(sentencia).split(" "))

# Eliminar URL y caracteres de nueva línea
def remove_three_dots(list_text):
    return [re.sub(r"[a-zA-Z]+(\……|\…)$", " ", texto) for texto in list_text]

def remove_three_dots(list_text):
    return [re.sub("[\.]"*3, " ", texto) for texto in list_text]

def remove_url(list_text):
    return [re.sub(r"http\S+", "", texto).strip() for texto in list_text]

def remove_breakline(list_text):
    return [re.sub('\s+', ' ', texto) for texto in list_text]

def remove_single_quotes(list_text):
    return [re.sub("\'", "", texto) for texto in list_text]


### Eliminar stoprwords, bigramas y lematizando texto
def remove_stopwords(texts, stop_words):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
def text_process_lema(data, diccionario):
    
    
    # Preparacion de palabras clave
    stop_words = stopwords.words('spanish')
    stop_words.extend(["rt", "aun", "oe"])
#     exclude = string.punctuation
#     exclude = exclude + "¿"
#     lemma = SnowballStemmer('spanish')

    print('Procesando textos (descripciones)...')
    
    # Texto a  minusculas
    data['texto']= data['Descripcion'].str.lower()
    print('Elementos nulos', len(data[data['texto'].isna()]))
    data['texto'] = data['texto'].fillna('')
    print('Elementos nulos', len(data[data['texto'].isna()]))
    txt_all = data['texto'].values.tolist()

    txt_all = remove_url(txt_all)
    txt_all = remove_three_dots(txt_all)
    txt_all = remove_breakline(txt_all)
    txt_all = remove_single_quotes(txt_all)

    data_palabras_all = list(texto_a_palabras(txt_all))

    bigram_all = gensim.models.Phrases(data_palabras_all, min_count=5, threshold=100)
    bigram_mod_all = gensim.models.phrases.Phraser(bigram_all)

    data_words_nostops_all = remove_stopwords(data_palabras_all, stop_words)
    data_words_bigrams_all = make_bigrams(data_words_nostops_all, bigram_mod_all)

    data_lemmatized_all = lemmatization(data_words_bigrams_all, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    diccionario_all = corpora.Dictionary(data_lemmatized_all)

    # Corpus
    texto_all = data_lemmatized_all
    corpus_all = [diccionario.doc2bow(doc) for doc in texto_all]
    return corpus_all